# Pipeline de Entrenamiento Autónomo: Agente Generador de Casos Clínicos (AI-Med)
Este notebook está diseñado para ejecutarse en Google Colab con una GPU T4. Entrenará un modelo Llama-3.1-8B para generar BattleCards basadas estrictamente en contexto médico extraído de NotebookLM.

In [ ]:
# MONTAR DRIVE AL INICIO
# Esto asegura que tus archivos no se pierdan si se desconecta el entorno.
from google.colab import drive
drive.mount('/content/drive')

import os
save_path = '/content/drive/MyDrive/AI_Med_Training'
if not os.path.exists(save_path):
    os.makedirs(save_path)
print(f'✅ Los resultados se guardarán automáticamente en: {save_path}')

## Celda 1: Preparación del Entorno Acelerado
Instalamos Unsloth y dependencias.

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes datasets

## Celda 2: Inicialización del Modelo Base Clínico
Cargamos `Llama-3.1-8B-Instruct` a 4 bits.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
load_in_4bit = True

# Modelo Base recomendado para tareas de estructuración profunda
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

## Celda 3: Configuración de Adaptadores LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 1234,
    use_rslora = True,
    loftq_config = None,
)

## Celda 4: Estructuración Estricta del Prompt y Dataset
⚠️ **IMPORTANTE:** Sube tu archivo `dataset_casos_clinicos.jsonl` a los archivos de Colab (panel izquierdo) antes de ejecutar esta celda.

In [ ]:
battlecard_prompt = """Eres el Agente Generador Experto de AI-Med Learning.
Tu ÚNICA tarea es extraer la información médica provista en el [Contexto Científico] (previamente extraído vía MCP de NotebookLM) y estructurarla de forma estricta en una [BattleCard] compatible con la plataforma del usuario.
Bajo NINGUNA circunstancia puedes inventar datos que no estén en el contexto. Si algo no está, no lo incluyas.

### Tema Solicitado:
{}

### Contexto Científico (Input desde MCP NotebookLM):
{}

### BattleCard (Salida Estructurada):
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    temas    = examples["tema"]
    contextos = examples["contexto_mcp"]
    salidas  = examples["casos_generados"]
    texts = []
    
    for tema, contexto, salida in zip(temas, contextos, salidas):
        text = battlecard_prompt.format(tema, contexto, salida) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
# Cargar la data estructurada de BattleCards
dataset = load_dataset("json", data_files="dataset_casos_clinicos.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

## Celda 5: Entrenamiento Especializado
Dará inicio al Fine-Tuning de los parámetros seleccionados con optimizador de 8-bit para cuidar la VRAM.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100, # Reducido un poco para recuperación rápida
        learning_rate = 1.5e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.05,
        lr_scheduler_type = "cosine",
        seed = 1234,
        output_dir = '/content/drive/MyDrive/AI_Med_Training/outputs', # GUARDADO AUTOMÁTICO
    ),
)

# Inicia la magia
trainer_stats = trainer.train()

## Celda 6: Validación Anti-Alucinación (Prueba de Cordura)
Forzamos al modelo a crear una tarjeta en base a un texto de prueba para ver cómo maneja el markdown en vivo.

In [ ]:
FastLanguageModel.for_inference(model)

tema_prueba = "Cetoacidosis Diabética (CAD)"
contexto_prueba = "Las guías 2024 recomiendan bolos de fluidos iniciales seguidos de insulina IV a 0.1 U/kg/hr. Criterios de resolución: Glucosa < 200, HCO3 >= 18."

inputs = tokenizer(
[
    battlecard_prompt.format(
        tema_prueba,
        contexto_prueba,
        "", # Dejamos este vacío para que la IA complete la respuesta estructuradamente
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, temperature=0.1, do_sample=False)

## Celda 7: Exportación a GGUF y Guardado en Google Drive
Fusiona los adaptadores, comprime a formato q4_k_m (ideal para Mac M3) y lo guarda directamente en Google Drive para evitar fallos por archivos grandes.

In [ ]:
# EXPORTACIÓN FINAL A DRIVE
# Como ya montamos el drive al inicio, esto es directo.
model.save_pretrained_gguf("AI_Med_Generator_V1", tokenizer, quantization_method = "q4_k_m")

import shutil
import os

try:
    gguf_file = [f for f in os.listdir("AI_Med_Generator_V1") if f.endswith(".gguf")][0]
    source_path = os.path.join("AI_Med_Generator_V1", gguf_file)
    destination_path = f"/content/drive/MyDrive/AI_Med_Training/{gguf_file}"
    
    print(f"Copiando {source_path} a tu Google Drive... (Esto tomará unos minutos)")
    shutil.copy2(source_path, destination_path)
    print(f"¡ÉXITO TOTAL! El archivo ya está seguro en tu Drive: AI_Med_Training/{gguf_file}")
except Exception as e:
    print(f"Error: {e}")